## _*Using Qiskit Aqua for max-cut problems*_

This Qiskit Aqua Optimization notebook demonstrates how to use the VQE quantum algorithm to compute the max cut of a given graph. 

The problem is defined as follows. Given a graph $G = (V,E)$ with weights $w_{ij}$ on the edges, we are looking for a subset $S \subseteq V$ such that $\sum_{(i,j) \in E : i \in S, j \not\in S} w_{ij}$ is maximized.

The graph provided as an input is used first to generate an Ising Hamiltonian, which is then passed as an input to VQE.  As a reference, this notebook also computes the max cut using the NumPyMinimumEigensolver classical algorithm and the solver embedded in the commercial non-quantum IBM CPLEX product (if it is available in the system and the user has followed the necessary configuration steps in order for Qiskit Aqua to find it).  Please refer to the Qiskit Aqua Optimization documentation for installation and configuration details for CPLEX.

In [1]:
import numpy as np

from qiskit import BasicAer
from qiskit.optimization.applications.ising import max_cut
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.circuit.library import TwoLocal
from qiskit.optimization.applications.ising.common import parse_gset_format, random_graph, sample_most_likely

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from an Ising Hamiltonian translated from the max-cut problem. We load a small sample instance of the max-cut problem.

In [2]:
w = parse_gset_format('sample.maxcut')
qubitOp, offset = max_cut.get_operator(w)

We also offer a function to generate a random graph as a input.

In [3]:
if True:
    np.random.seed(8123179)
    w = random_graph(4, edge_prob=0.5, weight_range=10)
    qubitOp, offset = max_cut.get_operator(w)
print(w)

[[ 0.  8. -9.  0.]
 [ 8.  0.  7.  9.]
 [-9.  7.  0. -8.]
 [ 0.  9. -8.  0.]]


Here we test for the presence of algorithms we want to use in this notebook. If Aqua is installed correctly `NumPyMinimumEigensolver` and `VQE` will always be found. `ClassicalCPLEX` is dependent on CPLEX being installed (see introduction above). CPLEX is *not required* but if installed then this notebook will demonstrate the `ClassicalCPLEX` algorithm , that uses CPLEX, to compute max-cut as well.

In [4]:
to_be_tested_algos = ['NumPyMinimumEigensolver', 'ClassicalCPLEX', 'VQE']
print(to_be_tested_algos)

['NumPyMinimumEigensolver', 'ClassicalCPLEX', 'VQE']


We can now use the Operator without regard to how it was created. First we need to prepare the configuration params to invoke the algorithm. Here we will use the NumPyMinimumEigensolver first to return the smallest eigenvalue. Backend is not required since this is computed classically not using quantum computation. We then add in the qubitOp Operator in dictionary format. The result is a dictionary.

In [5]:
result = NumPyMinimumEigensolver(qubitOp).run()
# print('objective function:', max_cut.max_cut_obj(result, offset))
x = sample_most_likely(result.eigenstate)
print('energy:', result.eigenvalue.real)
print('max-cut objective:', result.eigenvalue.real + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

energy: -20.5
max-cut objective: -24.0
solution: [1 0 1 1]
solution objective: 24.0


*Note*: If CPLEX nstalled then the Aqua CPLEX.Ising algorithm will be able to be used. If not, then solving this problem using this particular algorithm will simply be skipped. 

We change the configuration parameters to solve it with the CPLEX backend. The CPLEX backend can deal with a particular type of Hamiltonian called Ising Hamiltonian, which consists of only Pauli Z at most second order and often for combinatorial optimization problems that can be formulated as quadratic unconstrained binary optimization problems, such as the max-cut problem.

Note that for a max-cut problem, since we are computing a bipartition of the graph, every binary vector $x$ and its complement (i.e., the vector $y$ such that $y_j = 1 - x_j$ for all $j$) represent exactly the same solution, and will have the same objective function value. Different solution methods may return solutions that look different, but in fact have the same objective function value.

In [6]:
try:
    from qiskit.aqua.algorithms import ClassicalCPLEX
    result = ClassicalCPLEX(qubitOp, display=0).run()

    x_dict = result['x_sol']
    print('energy:', result['energy'])
    print('time:', result['eval_time'])
    print('max-cut objective:', result['energy'] + offset)
    x = np.array([x_dict[i] for i in sorted(x_dict.keys())])
    print('solution:', max_cut.get_graph_solution(x))
    print('solution objective:', max_cut.max_cut_value(x, w))
except Exception as ex:
    print(str(ex))

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             0
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Tolerances_MIPGap                   0
CPXPARAM_MIP_Tolerances_Integrality              0
energy: -20.5
time: 0.04090805899999994
max-cut objective: -24.0
solution: [1 0 1 1]
solution objective: 24.0


Now we want VQE and so change it and add its other configuration parameters.

In [7]:
vqe = VQE(qubitOp,
          TwoLocal(qubitOp.num_qubits, ['ry', 'rz'], 'cz', reps=3, entanglement='linear'),
          L_BFGS_B(maxfun=6000))
result = vqe.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))

x = sample_most_likely(result.eigenstate)
print('energy:', result.eigenvalue.real)
print('time:', result.optimizer_time)
print('max-cut objective:', result.eigenvalue.real + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

energy: -20.4999999999969
time: 5.653310775756836
max-cut objective: -23.9999999999969
solution: [0. 1. 0. 0.]
solution objective: 24.0
